# SPARTA (Semantic Parsing And Relational Table Aware)

This is a term project in `Unstructured Text Analysis` class.   
We implement the deep learning model for converting Korean language to SQL query. 

- github: https://github.com/TooTouch/SPARTA

<br>
<img src='https://user-images.githubusercontent.com/37654013/119700897-bec2c100-be8e-11eb-9d61-36de1ca66d5a.png'>
<br>

**Team Members**
- Hoonsang Yoon 
- Jaehyuk Heo 
- Jungwoo Choi
- Jeongseob Kim

**Information**
- Korea University [DSBA Lab](http://dsba.korea.ac.kr/)
- Advisor: [Pilsung Kang](http://dsba.korea.ac.kr/professor/)



In [28]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from konlpy.tag import Mecab

import os 
import sys
sys.path.append('../')

# from train import construct_hyper_param, \
#                   get_data, get_models, get_wemb_bert, \
#                   sort_and_generate_pr_w, generate_sql_q, \
#                   tokenize_corenlp_direct_version

In [33]:
from sqlova.utils.utils_wikisql import *
from sqlova.utils.utils import topk_multi_dim
from sqlnet.dbengine import DBEngine

import argparse
from IPython.display import display, clear_output
import ipywidgets

ModuleNotFoundError: No module named 'sqlova'

In [22]:
parser = argparse.ArgumentParser()
args = construct_hyper_param(parser, notebook=True)
device = 'cpu'

NameError: name 'argparse' is not defined

In [19]:
def inference(inputs, engine, datadir, bert_config, model_bert, tokenizer):
    nlu, nlu_t, hds, tb = inputs    
    
    # prediction
    wemb_n, wemb_h, l_n, l_hpu, l_hs, \
    nlu_tt, t_to_tt_idx, tt_to_t_idx = get_wemb_bert(bert_config, 
                                                     model_bert, 
                                                     tokenizer, 
                                                     nlu_t, 
                                                     hds, 
                                                     args.max_seq_length,
                                                     num_out_layers_n=args.num_target_layers, 
                                                     num_out_layers_h=args.num_target_layers,
                                                     device=device)

    prob_sca, prob_w, prob_wn_w, pr_sc, pr_sa, pr_wn, pr_sql_i, pr_sql_topk_i = model.beam_forward(wemb_n, 
                                                                                                   l_n, 
                                                                                                   wemb_h, 
                                                                                                   l_hpu,
                                                                                                   l_hs, 
                                                                                                   engine, 
                                                                                                   tb,
                                                                                                   nlu_t, 
                                                                                                   nlu_tt,
                                                                                                   tt_to_t_idx, 
                                                                                                   nlu,
                                                                                                   beam_size=args.beam_size,
                                                                                                   device=device)

    pr_wc, pr_wo, pr_wv, pr_sql_i = sort_and_generate_pr_w(pr_sql_i)

    pr_sql_q1 = generate_sql_q(pr_sql_i, tb)[0]
    pr_ans = engine.execute(tb[0]['id'], 
                            pr_sc[0], 
                            pr_sa[0], 
                            pr_sql_i[0]['conds'])
    
    return pr_sql_q1, pr_ans
    

In [20]:
args.datadir = '../data/ko_from_table'
args.logdir = '../logs/ko_from_table'
args.bert_name = 'bert-base-multilingual-cased'

In [27]:
_, _, _, _, test_data, test_table, _, _, test_loader = get_data(args.datadir, args)


# To start from the pre-trained models, un-comment following lines.
path_model_bert = os.path.join(args.logdir, 'model_bert_best.pt')
path_model = os.path.join(args.logdir, 'model_best.pt')
model, model_bert, tokenizer, bert_config = get_models(args, 
                                                       trained=True,
                                                       path_model_bert=path_model_bert, 
                                                       path_model=path_model, 
                                                       device=device)

NameError: name 'get_data' is not defined

In [14]:
engine = DBEngine(os.path.join(args.datadir, 'test.db'))

NameError: name 'args' is not defined

# Table information

In [30]:
def print_table_label(idx):
    
    tb = [test_table[test_data[idx]['table_id']]]
    hds = [tb[0]['header']]
    nlu_t = [test_data[idx]['question_tok']]
    nlu = [test_data[idx]['question']]
    sql = [test_data[idx]['sql']]

    # sql
    g_sql = generate_sql_q(sql, tb)[0]

    from_start_idx = g_sql.find('FROM')
    where_start_idx = g_sql.find('WHERE')
    if where_start_idx == -1:
        g_sql = g_sql[:from_start_idx] + '\n' + g_sql[from_start_idx:]
    else:
        g_sql = g_sql[:from_start_idx] + '\n' + g_sql[from_start_idx:where_start_idx] + '\n' + g_sql[where_start_idx:]

    # answer
    g_ans  = engine.execute(tb[0]['id'], 
                            sql[0]['sel'], 
                            sql[0]['agg'], 
                            sql[0]['conds'])[0]

    # plotting
    display(pd.DataFrame(tb[0]['rows'], columns=tb[0]['header']))

    print('='*20)
    print('Ground Truth')
    print('='*20)
    print('[Question]')
    print(nlu[0])
    print()
    print('[SQL]')
    print(g_sql)
    print()
    print('[RESULT]')
    print(g_ans)
    print('='*20)
    
    # inference
    display(ipywidgets.HTML('<h1>Inference</h1>'))
    question = ipywidgets.Text(description='QUESTION:', 
                           layout=ipywidgets.Layout(width='auto', display='flex'))
    button = ipywidgets.Button(description='TEST')
    output = ipywidgets.Output()
    display(ipywidgets.VBox([question, button]))
    display(output)
    
    def infer(b):
        q_tokenizer = Mecab()
        question_t = q_tokenizer.morphs(question.value)

        pr_sql_q1, pr_ans = inference(inputs=[[question.value], [question_t], hds, tb], 
                                      engine=engine,
                                      datadir=args.datadir, 
                                      bert_config=bert_config, 
                                      model_bert=model_bert, 
                                      tokenizer=tokenizer)
    
        # print results
        from_start_idx = pr_sql_q1.find('FROM')
        where_start_idx = pr_sql_q1.find('WHERE')
        if where_start_idx == -1:
            pr_sql_q1 = pr_sql_q1[:from_start_idx] + '\n' + pr_sql_q1[from_start_idx:]
        else:
            pr_sql_q1 = pr_sql_q1[:from_start_idx] + '\n' + pr_sql_q1[from_start_idx:where_start_idx] + '\n' + pr_sql_q1[where_start_idx:]
        
        with output:
            clear_output(wait=True)
            print('='*20)
            print('Inference')
            print('='*20)
            print('[SQL]')
            print(pr_sql_q1)
            print()
            print('[RESULT]')
            for ans in pr_ans:
                print(ans)
            print('='*20)
    
    button.on_click(infer)

In [31]:
def print_table_label(idx):
    
    tb = [test_table[test_data[idx]['table_id']]]
    hds = [tb[0]['header']]
    nlu_t = [test_data[idx]['question_tok']]
    nlu = [test_data[idx]['question']]
    sql = [test_data[idx]['sql']]

    # sql
    g_sql = generate_sql_q(sql, tb)[0]

    from_start_idx = g_sql.find('FROM')
    where_start_idx = g_sql.find('WHERE')
    if where_start_idx == -1:
        g_sql = g_sql[:from_start_idx] + '\n' + g_sql[from_start_idx:]
    else:
        g_sql = g_sql[:from_start_idx] + '\n' + g_sql[from_start_idx:where_start_idx] + '\n' + g_sql[where_start_idx:]

    # answer
    g_ans  = engine.execute(tb[0]['id'], 
                            sql[0]['sel'], 
                            sql[0]['agg'], 
                            sql[0]['conds'])[0]

    # plotting
    display(pd.DataFrame(tb[0]['rows'], columns=tb[0]['header']))

    print('='*20)
    print('Ground Truth')
    print('='*20)
    print('[Question]')
    print(nlu[0])
    print()
    print('[SQL]')
    print(g_sql)
    print()
    print('[RESULT]')
    print(g_ans)
    print('='*20)
    
    # inference
    display(ipywidgets.HTML('<h1>Inference</h1>'))
    question = ipywidgets.Text(description='QUESTION:', 
                           layout=ipywidgets.Layout(width='auto', display='flex'))
    button = ipywidgets.Button(description='TEST')
    output = ipywidgets.Output()
    display(ipywidgets.VBox([question, button]))
    display(output)
    
    def infer(b):
        q_tokenizer = Mecab()
        question_t = q_tokenizer.morphs(question.value)

        pr_sql_q1, pr_ans = inference(inputs=[[question.value], [question_t], hds, tb], 
                                      engine=engine,
                                      datadir=args.datadir, 
                                      bert_config=bert_config, 
                                      model_bert=model_bert, 
                                      tokenizer=tokenizer)
    
        # print results
        from_start_idx = pr_sql_q1.find('FROM')
        where_start_idx = pr_sql_q1.find('WHERE')
        if where_start_idx == -1:
            pr_sql_q1 = pr_sql_q1[:from_start_idx] + '\n' + pr_sql_q1[from_start_idx:]
        else:
            pr_sql_q1 = pr_sql_q1[:from_start_idx] + '\n' + pr_sql_q1[from_start_idx:where_start_idx] + '\n' + pr_sql_q1[where_start_idx:]
        
        with output:
            clear_output(wait=True)
            print('='*20)
            print('Inference')
            print('='*20)
            print('[SQL]')
            print(pr_sql_q1)
            print()
            print('[RESULT]')
            for ans in pr_ans:
                print(ans)
            print('='*20)
    
    button.on_click(infer)

In [32]:
index_text = ipywidgets.BoundedIntText(
    value=0,
    min=0,
    max=len(test_data),
    step=1,
    description='INDEX:',
    disabled=False
)

NameError: name 'ipywidgets' is not defined

In [38]:
ipywidgets.interactive(print_table_label, idx=index_text)

interactive(children=(BoundedIntText(value=0, description='INDEX:', max=15878), Output()), _dom_classes=('widg…

# Bridge Model Define

In [1]:
import src.utils.utils as utils
from src.semantic_parser.learn_framework import EncoderDecoderLFramework
from src.semantic_parser.ensemble_configs import model_dirs as ensemble_model_dirs
from src.eval.wikisql.lib.dbengine import DBEngine
import src.eval.eval_tools as eval_tools
from src.demos.demos import Text2SQLWrapper
from src.data_processor.path_utils import get_model_dir, get_checkpoint_path
from src.data_processor.schema_graph import SchemaGraph
from src.data_processor.vocab_processor import build_vocab
from src.data_processor.data_processor import preprocess
import src.data_processor.processor_utils as data_utils
import src.data_processor.data_loader as data_loader
import src.common.ops as ops
import random
import json
import os
import sys
import torch

from src.parse_args import args

In [2]:
args.data_dir="data/ko_token"
args.db_dir="data/ko_token"
args.dataset_name="wikisql"
args.model="bridge"
args.model_id = 2
args.question_split=True
args.query_split=False
args.question_only=True
args.normalize_variables=False
args.denormalize_sql=True
args.omit_from_clause=True
args.no_join_condition=False
args.table_shuffling=True
args.use_graph_encoding=False
args.use_typed_field_markers=False
args.use_lstm_encoder=True
args.use_meta_data_encoding=True
args.use_picklist=True
args.no_anchor_text=False
args.anchor_text_match_threshold=0.85
args.top_k_picklist_matches=2
args.atomic_value_copy=False
args.process_sql_in_execution_order=False
args.sql_consistency_check=False
args.share_vocab=False
args.sample_ground_truth=False
args.save_nn_weights_for_visualizations=True
args.vocab_min_freq=0
args.text_vocab_min_freq=0
args.program_vocab_min_freq=0
args.max_in_seq_len=512
args.max_out_seq_len=60

args.num_steps=10000
args.curriculum_interval=0
args.num_peek_steps=400
args.num_accumulation_steps=3
args.save_best_model_only=True
args.train_batch_size=8
args.dev_batch_size=8
args.encoder_input_dim=768
args.encoder_hidden_dim=512
args.decoder_input_dim=512
args.num_rnn_layers=1
args.num_const_attn_layers=0
args.use_oracle_tables=False
args.num_random_tables_added=0
args.use_additive_features=False
args.schema_augmentation_factor=1
args.random_field_order=False
args.data_augmentation_factor=1
args.augment_with_wikisql=False
args.num_values_per_field=0
args.pretrained_transformer="bert-base-multilingual-cased"
args.fix_pretrained_transformer_parameters=False
args.bert_finetune_rate=0.00005
args.learning_rate=0.0003
args.learning_rate_scheduler="inverse-square"
args.trans_learning_rate_scheduler="inverse-square"
args.warmup_init_lr=0.0003
args.warmup_init_ft_lr=0
args.num_warmup_steps=3000
args.emb_dropout_rate=0.3
args.pretrained_lm_dropout_rate=0
args.rnn_layer_dropout_rate=0.1
args.rnn_weight_dropout_rate=0
args.cross_attn_dropout_rate=0
args.cross_attn_num_heads=8
args.res_input_dropout_rate=0.2
args.res_layer_dropout_rate=0
args.ff_input_dropout_rate=0.4
args.ff_hidden_dropout_rate=0.0
args.grad_norm=0.3
args.decoding_algorithm="beam-search"
args.beam_size=8
args.bs_alpha=1.0
args.data_parallel=False

args.model_dir = './model/wikisql.bridge.lstm.meta.ts.ko_token.bs_8.ppl-0.85.2.dn.no_from.feat.bert-base-multilingual-cased.xavier-768-512-512-8-3-0.0003-inv-sqr-0.0003-3000-5e-05-inv-sqr-0.0-3000-0.3-0.3-0.0-0.0-1-8-0.1-0.0-res-0.2-0.0-ff-0.4-0.0.test'

In [3]:
sp = EncoderDecoderLFramework(args).cuda()

sp.load_checkpoint(get_checkpoint_path(args))
sp.eval()

* text vocab size = 119547
* program vocab size = 99

pretrained_transformer = bert-base-multilingual-cased
fix_pretrained_transformer_parameters = False

share_vocab is False
bridge module created


/home/hoonst/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


=> loading checkpoint './model/wikisql.bridge.lstm.meta.ts.ko_token.bs_8.ppl-0.85.2.dn.no_from.feat.bert-base-multilingual-cased.xavier-768-512-512-8-3-0.0003-inv-sqr-0.0003-3000-5e-05-inv-sqr-0.0-3000-0.3-0.3-0.0-0.0-1-8-0.1-0.0-res-0.2-0.0-ff-0.4-0.0.test/model-best.8.tar'


EncoderDecoderLFramework(
  (mdl): Bridge(
    (encoder_embeddings): TransformerHiddens(
      (trans_parameters): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelf

In [4]:
dataset = data_loader.load_processed_data(args)

split = 'test'
if args.dataset_name == 'wikisql':
    engine_path = os.path.join(args.data_dir, '{}.db'.format(split))
    engine = DBEngine(engine_path)
else:
    engine = None

loading preprocessed data: data/ko_token/wikisql.bridge.question-split.ppl-0.85.2.dn.no_from.bert.multilingual.ko_token.pkl


In [5]:
question_idx = 101

In [6]:
dataset['test'][question_idx].text

'초안에 LB이있는 college는 무엇입니까?'

In [7]:
import ast
import json

In [8]:
# Question with question_idx
question = '초안에 LB college가 뭐냐?'
dataset['test'][question_idx].text = question

examples = dataset[split][question_idx:question_idx+1]
sp.schema_graphs = dataset['schema']

table_id = examples[0].db_name

# define all tables
with open('./data/ko_from_table/test.tables.jsonl', 'r') as json_file:
    tables = list(json_file)
    
for i, table in enumerate(tables):
    tables[i] = ast.literal_eval(table)

print('{} {} examples loaded'.format(len(examples), split))

1 test examples loaded


In [9]:
pred_restored_cache = sp.load_pred_restored_cache()
pred_restored_cache_size = sum(len(v) for v in pred_restored_cache.values())

out_dict = sp.inference(examples, restore_clause_order=args.process_sql_in_execution_order,
                            pred_restored_cache=pred_restored_cache,
                            check_schema_consistency_=args.sql_consistency_check,
                            engine=engine, inline_eval=True, verbose=True)

100%|██████████| 1/1 [00:00<00:00,  3.88it/s]

In [11]:
pred_sql = [out_dict['pred_decoded'][0][-1]]
target_table = [table for table in tables if table["id"] == table_id]

In [13]:
from src.data_processor.processors.data_processor_wikisql import generate_sql_q, generate_sql_q1

In [14]:
generate_sql_q(pred_sql, target_table)

['SELECT (College) FROM 1-10812403-4 WHERE Position = LB']

In [15]:
sel = pred_sql[0]['sel']
agg = pred_sql[0]['agg']
conds = pred_sql[0]['conds']

engine.execute(table_id, sel, agg, conds)

['laval']